In [ ]:
#write a function that returns an array of color signals for each galaxy and an array of the redshift for each galaxy
import numpy as np
def get_features_targets(data):
    features = np.zeros(shape=(len(data), 4))
    features[:, 0] = data['u'] - data['g']
    features[:, 1] = data['g'] - data['r']
    features[:, 2] = data['r'] - data['i']
    features[:, 3] = data['i'] - data['z']
    targets = data['redshift']
    return features, targets

get_features_targets(np.load('sdss_galaxy_colors.npy'))

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

# copy in your get_features_targets function here
def get_features_targets(data):
    features = np.zeros(shape=(len(data), 4))
    features[:, 0] = data['u'] - data['g']
    features[:, 1] = data['g'] - data['r']
    features[:, 2] = data['r'] - data['i']
    features[:, 3] = data['i'] - data['z']
    targets = data['redshift']
    return features, targets

# load the data and generate the features and targets
features, targets = get_features_targets(np.load('sdss_galaxy_colors.npy'))
  
# initialize model
dtr = DecisionTreeRegressor()

# train the model
dtr.fit(features, targets)

# make predictions using the same features
predictions = dtr.predict(features)

# print out the first 4 predicted redshifts
print(predictions[:4])

In [ ]:
#write a function to calculate the median residual error of the model
import numpy as np

def median_diff(predicted, actual):
    return np.median(np.abs(predicted[:] - actual[:]))

targets = np.array([1,2,3,4])
predictions = np.array([1.1,2.1,3.1,4.1])

median_diff(predictions, targets)

In [1]:
# make a function that splits the model, trains it on the first half, then tests it on the second half
import numpy as np
from sklearn.tree import DecisionTreeRegressor

def get_features_targets(data):
    features = np.zeros(shape=(len(data), 4))
    features[:, 0] = data['u'] - data['g']
    features[:, 1] = data['g'] - data['r']
    features[:, 2] = data['r'] - data['i']
    features[:, 3] = data['i'] - data['z']
    targets = data['redshift']
    return features, targets

# paste your median_diff function here
def median_diff(predicted, actual):
    return np.median(np.abs(predicted - actual))

# splits data then trains and returns accuracy
def validate_model(model, features, targets):
    # split the data into training and testing
    # figures out a midpoint
    split = 2*features.shape[0]//3
    # splits the array down the midpoint
    train_features, test_features = features[:split], features[split:]
    train_targets, test_targets = targets[:split], targets[split:]

    # train the model
    model.fit(train_features, train_targets)

    # get the predicted_redshifts
    predictions = model.predict(test_features)  
  
    # use median_diff function to calculate the accuracy
    return median_diff(test_targets, predictions)


data = np.load('sdss_galaxy_colors.npy')
features, targets = get_features_targets(data)

# initialize model
dtr = DecisionTreeRegressor()

# validate the model and print the med_diff
validate_model(dtr, features, targets)

0.021853170000000005

In [ ]:
#make a plot of redshift in relation to colour
import numpy as np
from matplotlib import pyplot as plt

# Complete the following to make the plot
data = np.load('sdss_galaxy_colors.npy')
# Get a colour map
cmap = plt.get_cmap('YlOrRd')

# Define our colour indexes u-g and r-i
u_g = data['u'] - data['g']
r_i = data['r'] - data['i']
# Make a redshift array
redshift = data['redshift']
# Create the plot with plt.scatter and plt.colorbar
plot = plt.scatter(u_g, r_i, s=0.5, lw=0, c=redshift, cmap=cmap)
# Define your axis labels and plot title
plt.xlabel('Colour index  u-g')
plt.ylabel('Colour index  r-i')
plt.title('Redshift (colour) u-g versus r-i')
# Set any axis limits
plt.xlim(-0.5, 2.5)
plt.ylim(-0.5, 1)
plt.show()

In [ ]:
#test how accuracy changes with depth
import numpy as np
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeRegressor

def get_features_targets(data):
    features = np.zeros(shape=(len(data), 4))
    features[:, 0] = data['u'] - data['g']
    features[:, 1] = data['g'] - data['r']
    features[:, 2] = data['r'] - data['i']
    features[:, 3] = data['i'] - data['z']
    targets = data['redshift']
    return features, targets

# paste your median_diff function here
def median_diff(predicted, actual):
    return np.median(np.abs(predicted - actual))

# Complete the following function
def accuracy_by_treedepth(features, targets, depths):
    # split the data into testing and training sets
    split = features.shape[0]//2
    train_features, test_features = features[:split], features[split:]
    train_targets, test_targets = targets[:split], targets[split:]

    # initialise arrays or lists to store the accuracies for the below loop
    train_diffs = []
    test_diffs = []
    # loop through depths
    for depth in depths:
        # initialize model with the maximum depth. 
        dtr = DecisionTreeRegressor(max_depth=depth)

        # train the model using the training set
        dtr.fit(train_features, train_targets)
        # get the predictions for the training set and calculate their median_diff
        predictions = dtr.predict(train_features)
        train_diffs.append(median_diff(train_targets, predictions))
        # get the predictions for the testing set and calculate their median_diff
        predictions = dtr.predict(test_features)
        test_diffs.append(median_diff(test_targets, predictions))

    # return the accuracies for the training and testing sets
    return train_diffs, test_diffs

data = np.load('sdss_galaxy_colors.npy')
features, targets = get_features_targets(data)

# Generate several depths to test
tree_depths = range(1, 36, 2)

# Call the function
train_diffs, test_diffs = accuracy_by_treedepth(features, targets, tree_depths)
print("Depth with lowest median difference : {}".format(tree_depths[test_diffs.index(min(test_diffs))]))
    
# Plot the results
train_plot = plt.plot(tree_depths, train_diffs, label='Training set')
test_plot = plt.plot(tree_depths, test_diffs, label='Validation set')
plt.xlabel("Maximum Tree Depth")
plt.ylabel("Median of Differences")
plt.legend()
plt.show()


In [2]:
# do the same thing as in model validation but with kfold cross validation rather than hold out
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

def get_features_targets(data):
    features = np.zeros(shape=(len(data), 4))
    features[:, 0] = data['u'] - data['g']
    features[:, 1] = data['g'] - data['r']
    features[:, 2] = data['r'] - data['i']
    features[:, 3] = data['i'] - data['z']
    targets = data['redshift']
    return features, targets

def median_diff(predicted, actual):
    return np.median(np.abs(predicted - actual))

def cross_validate_model(model, features, targets, k):
    kf = KFold(n_splits=k, shuffle=True)

    # initialise a list to collect median_diffs for each iteration of the loop below
    diffs = []

    for train_indices, test_indices in kf.split(features):
        train_features, test_features = features[train_indices], features[test_indices]
        train_targets, test_targets = targets[train_indices], targets[test_indices]
    
        # fit the model for the current set
        model.fit(train_features, train_targets)
    
        # predict using the model
        predictions = model.predict(test_features)
 
        # calculate the median_diff from predicted values and append to results array
        diffs.append(median_diff(predictions, test_targets))
 
    # return the list with your median difference values
    return diffs


data = np.load('sdss_galaxy_colors.npy')
features, targets = get_features_targets(data)

# initialize model with a maximum depth of 19
dtr = DecisionTreeRegressor(max_depth=21)

# call your cross validation function
diffs = cross_validate_model(dtr, features, targets, 10)

# Print the values
print('Differences: {}'.format(', '.join(['{:.3f}'.format(val) for val in diffs])))
print('Mean difference: {:.3f}'.format(np.mean(diffs)))

Differences: 0.018, 0.018, 0.018, 0.017, 0.017, 0.017, 0.018, 0.017, 0.017, 0.018
Mean difference: 0.018


In [ ]:
# just return predictions, again using cross fold
import numpy as np
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor

def get_features_targets(data):
    features = np.zeros(shape=(len(data), 4))
    features[:, 0] = data['u'] - data['g']
    features[:, 1] = data['g'] - data['r']
    features[:, 2] = data['r'] - data['i']
    features[:, 3] = data['i'] - data['z']
    targets = data['redshift']
    return features, targets


def cross_validate_predictions(model, features, targets, k):
    kf = KFold(n_splits=k, shuffle=True)
    all_predictions = np.zeros_like(targets)
    for train_indices, test_indices in kf.split(features):
        train_features, test_features = features[train_indices], features[test_indices]
        train_targets, test_targets = targets[train_indices], targets[test_indices]
    
        # fit the model for the current set
        model.fit(train_features, train_targets)
    
        # predict using the model
        predictions = model.predict(test_features)
        all_predictions[test_indices] = predictions
    # return the list with your median difference values
    return all_predictions

data = np.load('sdss_galaxy_colors.npy')
features, targets = get_features_targets(data)

# initialize model with a maximum depth of 19
dtr = DecisionTreeRegressor(max_depth=19)

# call your cross validation function
cross_validate_predictions(dtr, features, targets, 10)